In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install transformers
!pip install swifter
!pip install dgl-cu100
!pip install pytorch-metric-learning

In [0]:
import pandas as pd
import numpy as np
import datetime
import random
import os
import time
import gc
from glob import glob
from tqdm import tqdm_notebook as tqdm
import pickle
import logging
import json

import base64
import swifter
import gensim

from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
from sklearn.utils import shuffle
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
import matplotlib.pyplot as plt
#import japanize_matplotlib
import seaborn as sns

import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader, Dataset

pd.set_option('display.max_columns', 500)

In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset

import keras
from keras.preprocessing.sequence import pad_sequences

import networkx as nx
import dgl
import dgl.function as fn
from dgl.nn.pytorch import GraphConv

from pytorch_metric_learning import losses

import transformers
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,DistilBertTokenizer,DistilBertModel
)

print(transformers.__version__)

In [0]:
os.getcwd()

In [0]:
DATA_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/data/'
LOG_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/log/'
W2V_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/w2v/'
MODEL_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/model/'
SUBMIT_DIR = '/content/drive/My Drive/Colab Notebooks/KDDCUP/prediction_result/'

In [0]:
# DATA_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/data/'
# LOG_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/log/'
# W2V_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/w2v/'
# BERT_DIR = 'D:/project/ICF_AutoCapsule_disabled/BERT/'
# MODEL_DIR = 'D:/project/ICF_AutoCapsule_disabled/kddcup2020/model/'

### Read Data

In [0]:
train = pd.read_csv(DATA_DIR+'train/train.sample.tsv', sep='\t')
print(train.shape)
train.head()

In [0]:
valid = pd.read_csv(DATA_DIR+'valid/valid.tsv', sep='\t')
print(valid.shape)
valid.head()

In [0]:
testA = pd.read_csv(DATA_DIR+'testA/testA.tsv', sep='\t')
print(testA.shape)
testA.head()

### Decode Data

In [0]:
def transform_boxes(df_row):
    return np.frombuffer(base64.b64decode(df_row.boxes), dtype=np.float32).reshape(df_row.num_boxes, 4)

def transform_features(df_row):
    return np.frombuffer(base64.b64decode(df_row.features), dtype=np.float32).reshape(df_row.num_boxes, 2048)

def transform_class_labels(df_row):
    return np.frombuffer(base64.b64decode(df_row.class_labels), dtype=np.int64).reshape(df_row.num_boxes)

def transform_df(df):
    df.boxes = df.swifter.apply(transform_boxes, axis=1)
    df.features = df.swifter.apply(transform_features, axis=1)
    df.class_labels = df.swifter.apply(transform_class_labels, axis=1)
    return df

In [0]:
train = transform_df(train)
valid = transform_df(valid)
testA = transform_df(testA)

In [0]:
train.head()

### Feature Engineering

- box area

In [0]:
train['box_normalized'] = train.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)
valid['box_normalized'] = valid.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)
testA['box_normalized'] = testA.swifter.apply(lambda x: np.array([[xi[0]/x.image_h,xi[1]/x.image_w,xi[2]/x.image_h,xi[3]/x.image_w] for xi in x.boxes]), axis=1)

In [0]:
train.head()

In [0]:
# box area
train['box_area_normalized'] = train.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))
valid['box_area_normalized'] = valid.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))
testA['box_area_normalized'] = testA.box_normalized.apply(lambda x: np.array([(xi[3]-xi[1]) * (xi[2]-xi[0]) for xi in x]))

In [0]:
train.head()

- word2vec of box labels

In [0]:
class_label_dict = {
    0:'top clothes (coat, jacket, shirt, etc.)',
    1:'skirt & dress',
    2:'bottom clothes (trousers, pants, etc.)',
    3:'luggage, leather goods',
    4:'shoes',
    5:'accessories (jewelry, clothing accessories, belts, hats, scarves, etc.)',
    6:'snacks, nuts, liquor and tea',
    7:'makeup, perfume, beauty tools and essential oils',
    8:'bottle drink',
    9:'furniture',
    10:'stationery',
    11:'household electrical appliances',
    12:'home decoration',
    13:'household fabric',
    14:'kitchenware',
    15:'home / personal cleaning tools',
    16:'storage supplies',
    17:'motorcycle, motorcycle accessories, vehicles, bicycle and riding equipment',
    18:'outdoor product',
    19:'lighting',
    20:'toys',
    21:'underwear',
    22:'digital supplies',
    23:'bed linens',
    24:'baby products',
    25:'personal care',
    26:'sporting goods',
    27:'clothes (accessories, baby clothing, etc.)',
    28:'others',
    29:'human face',
    30:'arm',
    31:'hair',
    32:'hand',
}

train.class_labels = train.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
valid.class_labels = valid.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))
testA.class_labels = testA.class_labels.swifter.apply(lambda x: (' ').join([class_label_dict[xi] for xi in x]))

In [0]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')

def load_embeddings(path):
    with open(path, encoding='utf-8') as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():  # word_indexのwordに対応するembがあれば、embを代入する
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words


def load_word_embed(word_embed_glove=W2V_DIR+"glove.840B.300d.txt", 
                    word_embed_crawl=W2V_DIR+"crawl-300d-2M.vec",
                    save_filename=W2V_DIR+"word_embedding_matrix_testA",
                    word_index=None):
    
    # Tokneizerの学習
    Tokenizer = keras.preprocessing.text.Tokenizer(filters='', lower=False)
    Tokenizer.fit_on_texts(list(train['query'])+list(valid['query'])+list(testA['query'])
                           +list(train['class_labels'])+list(valid['class_labels'])+list(testA['class_labels']))
    
    if os.path.exists(save_filename + ".npy"):
        embedding_matrix = np.load(save_filename + ".npy").astype("float32")
    else:
        
        if word_index is None:
            word_index = Tokenizer.word_index
        
        glove_matrix, unknown_words_glove = build_matrix(word_index, word_embed_glove)
        print('n unknown words (glove): ', len(unknown_words_glove))
        
        crawl_matrix, unknown_words_crawl = build_matrix(word_index, word_embed_crawl)
        print('n unknown words (crawl): ', len(unknown_words_crawl))
        
        embedding_matrix = crawl_matrix + glove_matrix  
        np.save(save_filename, embedding_matrix)

        del crawl_matrix
        del glove_matrix
        gc.collect()
        
    return embedding_matrix, Tokenizer


def calc_mean_w2v(class_labels):
    class_labels_index = tokenizer_w2v.texts_to_sequences([class_labels])
    return np.mean(embedding_matrix[class_labels_index], axis=0)
    
    # try:
    #     return embedding_matrix[class_labels_index]
    #     #return np.mean(embedding_matrix[class_labels_index], axis=0)
    # except KeyError:
    #     print('Error')
    #     return np.zeros(300)

In [0]:
# about 5 minites
# embedding_matrix, tokenizer_w2v = load_word_embed()

In [0]:
# train['class_labels_vec'] = train['class_labels'].swifter.apply(calc_mean_w2v)
# valid['class_labels_vec'] = valid['class_labels'].swifter.apply(calc_mean_w2v)
# testA['class_labels_vec'] = testA['class_labels'].swifter.apply(calc_mean_w2v)

- Adjaccency matrix
  - 面積が一番大きいboxを中心に、それと他の全ての結合グラフを考える

In [0]:
# from numba import jit

# @jit
# def get_adj_mat(X, Y, adj_mat):
#     n_box = adj_mat.shape[0]
#     if n_box!=1:
#         for i in range(n_box):
#             for j in range(i+1, n_box):
#                 d = np.sqrt((X[i] - X[j])**2 + (Y[i] - Y[j])**2)
#                 adj_mat[i, j] = 1/d
#                 adj_mat[j, i] = 1/d
                
#     return adj_mat

# def get_adj_mat_row(x):
#     n_box = x.num_boxes
#     adj_mat = np.zeros((n_box, n_box))
#     return get_adj_mat(x.center_normalized[:,0], x.center_normalized[:,1], adj_mat)

In [0]:
# # top/left/bottom/right
# train['center_normalized'] = train.box_normalized.apply(lambda x: np.array([[((xi[3]-xi[1])/2+xi[1]), ((xi[2]-xi[0])/2+xi[0])] for xi in x]))
# valid['center_normalized'] = valid.box_normalized.apply(lambda x: np.array([[((xi[3]-xi[1])/2+xi[1]), ((xi[2]-xi[0])/2+xi[0])] for xi in x]))
# testA['center_normalized'] = testA.box_normalized.apply(lambda x: np.array([[((xi[3]-xi[1])/2+xi[1]), ((xi[2]-xi[0])/2+xi[0])] for xi in x]))

In [0]:
train.head()

In [0]:
# train['adj_mat'] = train.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)
# valid['adj_mat'] = valid.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)
# testA['adj_mat'] = testA.swifter.apply(lambda x: get_adj_mat_row(x), axis=1)

In [0]:
train.head()

### Negative Sample生成
- ここでは、簡単のため、queryだけランダムにシャッフルしたデータフレームを作成し、concatする
- ラベルとして、元データはlabel=1, ランダムシャッフルはlabel=0とする

In [0]:
train['label'] = 1

random.seed(42)
ind = list(train.index)
random.shuffle(ind)

train_neg = train.copy(deep=True)
train_neg['label'] = 0
train_neg['query'] = list(train_neg['query'].loc[ind])[:]
train_neg.head()

train = pd.concat([train, train_neg], axis=0)
train = train.reset_index(drop=True)
print(train.shape)

In [0]:
train.iloc[0]

In [0]:
train.iloc[10000]

## Model

- initial setting

In [0]:
# make logger
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
sc = logging.StreamHandler()
logger.addHandler(sc)
fh = logging.FileHandler(LOG_DIR+'20200503_01.log')
logger.addHandler(fh)

In [0]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

### DataLoader

In [0]:
MAX_LEN = 20

class KDDDataset(Dataset):

    def __init__(self, df, train_mode=True, transform=None):
        self.df = df
        self.bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.w2v_tokenizer = tokenizer_w2v
        self.train_mode = train_mode
        
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        ######################
        # query side
        ######################
        query = row['query']
        inputs_query = self.bert_tokenizer.encode_plus(
            query,
            add_special_tokens=True,
            max_length=MAX_LEN,
        )
        
        ids_query = inputs_query["input_ids"]
        token_type_ids_query = inputs_query["token_type_ids"]
        mask_query = inputs_query["attention_mask"]
        
        padding_len = MAX_LEN - len(ids_query)
        ids_query = ids_query + ([0]*padding_len)
        token_type_ids_query = token_type_ids_query + ([0]*padding_len)
        mask_query = mask_query + ([0]*padding_len)
        
        
        ######################
        # image side
        ######################
        # box_areaの大きい順にソート
        box_order = np.argsort(row.box_area_normalized)[::-1]

        faetures_ordered = row.features[box_order, :]
        box_ordered = row.box_normalized[box_order, :]
        area_ordered = row.box_area_normalized[box_order]

        n_box = row.num_boxes
        adj_mat = np.zeros((n_box, n_box))
        if n_box!=1:
            X = box_ordered[:,0]
            Y = box_ordered[:,1]
            for i in range(1, n_box):
                d = np.sqrt((X[0] - X[i])**2 + (Y[0] - Y[i])**2)
                adj_mat[0, i] = 1/d
                adj_mat[i, 0] = 1/d

        # adj_mat = row['adj_mat']
        # features = row['features']
        # box = row['box_normalized']
        # box_area = row['box_area_normalized']
        
        nxg = nx.from_numpy_matrix(adj_mat)
        g = dgl.DGLGraph()
        g.from_networkx(nxg)
        g.edata['weight'] = torch.tensor(nx.adjacency_matrix(nxg).data, dtype=torch.float).to(device)
        g.ndata['h'] = torch.cat([torch.tensor(faetures_ordered, dtype=torch.float),torch.tensor(box_ordered, dtype=torch.float),torch.tensor(area_ordered.reshape(-1,1), dtype=torch.float)], dim=-1).to(device)
        g.ndata['w'] = torch.tensor(area_ordered, dtype=torch.float).to(device)
        
        # class_labels
        #class_labels_vec = row['class_labels_vec']
        
        
        if self.train_mode:
            # positive, negative label
            labels = row['label']
            
            
            return g, {
                'ids_query': torch.tensor(ids_query, dtype=torch.long),
                'mask_query': torch.tensor(mask_query, dtype=torch.long),
                'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
                #'class_labels_vec': torch.tensor(class_labels_vec, dtype=torch.float),
                'label': torch.tensor(labels, dtype=torch.float),
            }
        
        else:
            return g, {
                'ids_query': torch.tensor(ids_query, dtype=torch.long),
                'mask_query': torch.tensor(mask_query, dtype=torch.long),
                'token_type_ids_query': torch.tensor(token_type_ids_query, dtype=torch.long),
                #'class_labels_vec': torch.tensor(class_labels_vec, dtype=torch.float),
            }

In [0]:
def collate(samples):
    # input samples は、ペアのリスト
    #  [(graph, {features}),...], バッチサイズの長さのリスト
    # グラフは、隣接行列を対角につなげて1つのバッチにする
    # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
    graphs, features_dict = map(list, zip(*samples))
    
    batched_graph = dgl.batch(graphs)
    
    batched_features = {}
    labels = []
    for i, feat in enumerate(features_dict):
        if i==0:
            for (key, val) in feat.items():
                if key!='label':
                    batched_features[key] = val.view(1,-1)
                else:
                    labels.append(val.item())
        else:
            for (key, val) in feat.items():
                if key!='label':
                    batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
                else:
                    labels.append(val.item())
    
    batched_features['label'] = torch.tensor(labels, dtype=torch.float)
    
    return batched_graph, batched_features


def collate_test(samples):
    # input samples は、ペアのリスト
    #  [(graph, {features}),...], バッチサイズの長さのリスト
    # グラフは、隣接行列を対角につなげて1つのバッチにする
    # 他のfeatureは、(バッチサイズ, 次元数)のtensor
    
    graphs, features_dict = map(list, zip(*samples))
    
    batched_graph = dgl.batch(graphs)
    
    batched_features = {}
    labels = []
    for i, feat in enumerate(features_dict):
        if i==0:
            for (key, val) in feat.items():
                if key!='label':
                    batched_features[key] = val.view(1,-1)
                else:
                    labels.append(val.item())
        else:
            for (key, val) in feat.items():
                if key!='label':
                    batched_features[key] = torch.cat([batched_features[key],  val.view(1,-1)], dim=0)
                else:
                    labels.append(val.item())
        
    return batched_graph, batched_features

In [0]:
# # dataloaderの挙動確認

# dataset_train = KDDDataset(train, train_mode=True)
# train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)

# for i, (graph, features) in enumerate(train_loader):
#     if i<1:
#         print(graph.batch_size)
#         print(graph)
#         print(features['ids_query'].shape)
#         print(features['mask_query'].shape)
#         print(features['token_type_ids_query'].shape)
#         print(features['class_labels_vec'].shape)
#         print(features['label'].shape)

#     else:
#         break

## Model

In [0]:
class GCN(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(GCN, self).__init__()
        self.fc = nn.Linear(in_feats, out_feats)
        self.activation = activation
    
    def message_func(self, edges):
        """edgeを通して，どのようなメッセージをどう渡すかを決める関数"""
        weight = edges.data['weight'].view(-1, 1)
        messages = edges.src['h'] * weight
        return {'m': messages}
    
    def reduce_func(self, nodes):
        """渡されたメッセージをどのように集約するかを決める関数"""
        messages = nodes.mailbox['m']
        h = torch.sum(messages, dim=1)
        return {'h': h}
    
    def node_apply_func(self, nodes):
        """ノードごとに作用させる関数"""
        h = self.fc(nodes.data['h'])
        h = self.activation(h)
        return {'h': h}

    def forward(self, g, h):
        g.ndata['h'] = h
        g.apply_nodes(self.node_apply_func)
        g.update_all(self.message_func, self.reduce_func)
        return g.ndata.pop('h')  # ndata['h']が削除されるが，返り値としては，ndata['h']となる

In [0]:
class QueryModel(nn.Module):
    def __init__(self):
        super(QueryModel, self).__init__()
        self.model_name = 'QueryModel'

        # query
        self.bert_model = BertModel.from_pretrained('bert-base-uncased')
        self.fc_bert = nn.Linear(768, 128)
        self.fc_bert2 = nn.Linear(128, 64)


    def forward(self, ids, token_type_ids, mask):
        
        ##############################
        # query
        ##############################
        layers, pool_out = self.bert_model(input_ids=ids, token_type_ids=token_type_ids, attention_mask=mask)
        out_query = F.avg_pool1d(layers.transpose(1,2), kernel_size=layers.size()[1]).squeeze()
        out_query = F.dropout(out_query, p=0.3, training=self.training)
        out_query = F.relu(self.fc_bert(out_query))
        out_query = F.relu(self.fc_bert2(out_query))
        
        return out_query

In [0]:
class ImageModel(nn.Module):
    def __init__(self):
        super(ImageModel, self).__init__()
        self.model_name = 'ImageModel'

        # image
        self.gcn1 = GCN(2048+5, 512, F.relu)
        self.gcn2 = GCN(512, 256, F.relu)
        #self.fc1 = nn.Linear(256+300, 256)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 64)


    def forward(self, g):
        
        ##############################
        # image
        ##############################
        h = g.ndata.pop('h')
        h = self.gcn1(g, h)
        h = self.gcn2(g, h)
        g.ndata['h'] = h
        
        for i, graph in enumerate(dgl.unbatch(g)):
            if i==0:
                out_image = graph.ndata['h'][0].unsqueeze(0)
            else:
                out_image = torch.cat([out_image, graph.ndata['h'][0].unsqueeze(0)], dim=0)


        # Calculate graph representation by averaging all the node representations.
        #out_image = dgl.mean_nodes(g, 'h', 'w')  # wでの重み付き平均
        # out_image = dgl.max_nodes(g, 'h')
        
        #out_image = torch.cat([out_image, class_labels_vec], dim=-1)
        
        out_image = F.dropout(out_image, p=0.3, training=self.training)
        out_image = F.relu(self.fc1(out_image))
        out_image = F.dropout(out_image, p=0.3, training=self.training)
        out_image = F.relu(self.fc2(out_image))

        
        return out_image

In [0]:
class KDDModel(nn.Module):
    def __init__(self):
        super(KDDModel, self).__init__()
        self.model_name = 'KDDModel'

        self.model_query = QueryModel()
        self.model_image = ImageModel()


    def forward(self, ids, token_type_ids, mask, g):
        
        out_query = self.model_query(ids, token_type_ids, mask)
        out_image = self.model_image(g, class_labels_vec)

        out = (out_query * out_image).squeeze()
        #print(out.shape)
        
        return out.squeeze()

### Training

In [0]:
def train_model(train_loader, model, optimizer, criterion): #, scheduler):
    model.train()
    avg_loss = 0.
    avg_score = 0.
    for idx, (graph, features) in enumerate(tqdm(train_loader)):
        ids_query = features['ids_query'].to(device)
        mask_query = features['mask_query'].to(device)
        token_type_ids_query = features['token_type_ids_query'].to(device)
        labels = features['label'].to(device)

        out = model(ids_query, token_type_ids_query, mask_query, graph)
        
        loss = criterion(out, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        avg_loss += loss.item() / len(train_loader)

        del ids_query,mask_query,token_type_ids_query,class_labels_vec,labels,out,loss

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss



def test_model(test_loader, model):
    model.eval()
    
    out_list = []   
    with torch.no_grad():
        for idx, (graph, features) in enumerate(tqdm(test_loader)):
            ids_query = features['ids_query'].to(device)
            mask_query = features['mask_query'].to(device)
            token_type_ids_query = features['token_type_ids_query'].to(device)

            out_query = model.model_query(ids_query, token_type_ids_query, mask_query)
            out_image = model.model_image(graph, class_labels_vec)
            out = F.cosine_similarity(out_query, out_image)

            out_list.append(out)
            del ids_query,mask_query,token_type_ids_query,out
        
    torch.cuda.empty_cache()
    gc.collect()
    return out_list

In [0]:
# compute dcg@k for a single sample
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(3, r.size + 2)))
    return 0.


# compute ndcg@k (dcg@k / idcg@k) for a single sample
def get_ndcg(r, ref, k):
    dcg_max = dcg_at_k(ref, k)
    if not dcg_max:
        return 0.
    dcg = dcg_at_k(r, k)
    return dcg / dcg_max


def make_score(df_test, out_list):
    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    predictions = {}
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        predictions[f'{q_id}'] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values.astype(str)
    
    
    # read ground-truth
    reference = json.load(open(DATA_DIR+'valid/valid_answer.json'))

    # compute score for each query
    k = 5
    score_sum = 0.
    for qid in reference.keys():
        ground_truth_ids = set([str(pid) for pid in reference[qid]])
        ref_vec = [1.0] * len(ground_truth_ids)
        pred_vec = [1.0 if pid in ground_truth_ids else 0.0 for pid in predictions[qid]]
        score_sum += get_ndcg(pred_vec, ref_vec, k)
    # the higher score, the better
    score = score_sum / len(reference)
    
    return score

In [0]:
model = KDDModel().to(device)

# if device == 'cuda':
#     model = torch.nn.DataParallel(model) # make parallel
#     cudnn.benchmark = True


optimizer = torch.optim.Adam(model.parameters())

#criterion = nn.BCEWithLogitsLoss()
#criterion = nn.CosineSimilarity()
#criterion = nn.CosineEmbeddingLoss()
criterion = losses.TripletMarginLoss(margin=0.1)

loss_list_epoch_train = []
score_list_epoch_valid = []
score_best = 0
patience = 0
for epoch in range(20):

    torch.cuda.empty_cache()
    start_time   = time.time()

    dataset_train = KDDDataset(train, train_mode=True)
    dataset_valid = KDDDataset(valid, train_mode=False)
    train_loader = DataLoader(dataset_train, batch_size=128, shuffle=True, num_workers=0, drop_last=True, collate_fn=collate)
    valid_loader = DataLoader(dataset_valid, batch_size=128, shuffle=False, num_workers=0, drop_last=True, collate_fn=collate_test)
    #scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=config.warmup, num_training_steps=config.epochs*len(train_loader))

    loss_train = train_model(train_loader, model, optimizer, criterion)#, scheduler)
    out_valid = test_model(valid_loader, model)
    score_valid = make_score(valid, out_valid)

    loss_list_epoch_train.append(loss_train)
    score_list_epoch_valid.append(score_valid)

    logger.info(f'Epoch {(epoch+1)}, train_loss: {loss_train}, valid_score: {score_valid}, time: {(time.time()-start_time)}')

    # Eearly Stopping
    if score_valid > score_best:
        score_best = score_valid
        best_param = model.state_dict()
        torch.save(best_param, MODEL_DIR+f'best_param.pt')
        patience = 0
    else:
        patience += 1
        if patience >= 2:
            del train_loader, valid_loader, loss_train, out_valid
            torch.cuda.empty_cache()
            gc.collect()
            break

    del train_loader, valid_loader, loss_train, out_valid
    torch.cuda.empty_cache()
    gc.collect()

## Inference

In [0]:
model = KDDModel().to(device)
model.load_state_dict(torch.load(MODEL_DIR+f'best_param.pt'))

In [0]:
def make_submit_df(df_test, model_query, model_image):
    dataset_test = KDDDataset(df_test, train_mode=False)
    test_loader = DataLoader(dataset_test, batch_size=128, shuffle=False, num_workers=0, drop_last=True, collate_fn=collate_test)
    out_list = test_model(test_loader, model_query, model_image)

    for i, out in enumerate(out_list):
        if i==0:
            out_array = out.squeeze()
        else:
            out_array = torch.cat([out_array, out.squeeze()], dim=-1)

    out_array = out_array.cpu().detach().numpy()
    df_test['score'] = out_array
    
    # sort and group by
    df_test = df_test.sort_values(by="score", ascending=False)
    grouped = df_test.groupby("query_id").head(5)
    
    submit_df = pd.DataFrame(columns=['query-id','product1','product2','product3','product4','product5'])
    for i, q_id in enumerate(tqdm(grouped['query_id'].unique())):
        submit_df.loc[i, 'query-id'] = q_id
        submit_df.iloc[i, 1:] = grouped.loc[(grouped['query_id']==q_id), 'product_id'].values
    submit_df = submit_df.astype(int)
    submit_df = submit_df.sort_values(by='query-id')
    submit_df = submit_df.reset_index(drop=True)
    
    return submit_df

In [0]:
submit_df = make_submit_df(testA, model)
submit_df.to_csv(SUBMIT_DIR+'submission.csv', index=False)